# TP : Prédiction des frais d'assurance santé avec la régression et les KNN

## Objectif
Dans ce TP, vous allez explorer un jeu de données contenant des informations sur des personnes et leurs frais d'assurance. L'objectif est de prédire ces frais en utilisant des modèles de régression : régression linéaire et k plus proches voisins (KNN). Vous allez également analyser l'impact du nombre de voisins sur la performance du modèle KNN.

# 1. Importation des bibliothèques
Importer les bibliothèques nécessaires pour manipuler les données, les visualiser et construire les modèles de machine learning.

In [1]:
import pandas as pd

# 2. Chargement des données
Charger les données depuis le fichier insurance.csv disponible sur moodle.

In [ ]:
df = pd.read_csv('insurance.csv')

# 3. Exploration des données
Analyser les premières lignes du dataset et quelques statistiques générales.

## Afficher les premières lignes du dataset

In [ ]:
df.head(10)

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
5,31,female,25.740,0,no,southeast,3756.62160
6,46,female,33.440,1,no,southeast,8240.58960
7,37,female,27.740,3,no,northwest,7281.50560
8,37,male,29.830,2,no,northeast,6406.41070
9,60,female,25.840,0,no,northwest,28923.13692


## Afficher des statistiques générales sur les variables numériques

In [ ]:
df.describe()

,age,bmi,children,charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010


## Vérifier les valeurs manquantes

In [ ]:
df.isnull()

,age,sex,bmi,children,smoker,region,charges
0,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...
1333,False,False,False,False,False,False,False
1334,False,False,False,False,False,False,False
1335,False,False,False,False,False,False,False
1336,False,False,False,False,False,False,False


## Visualiser la distribution de la variable cible (charges)

In [ ]:
df.isna().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

# 4. Prétraitement des données
### Encodage des variables catégorielles
Transformer les variables catégorielles en variables numériques avec `OrdinalEncoder`.

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder()

categorical_columes = ['sex', 'smoker', 'region']

encoded_features = enc.fit_transform(df[categorical_columes])
print(encoded_features)

encoded_df = pd.DataFrame(encoded_features, columns=categorical_columes)
encoded_df.head(10)

df[['sex','smoker','region']] = enc.fit_transform(df[['sex','smoker','region']])
df.head(10)

[[0. 1. 3.]
 [1. 0. 2.]
 [1. 0. 2.]
 ...
 [0. 0. 2.]
 [0. 0. 3.]
 [0. 1. 1.]]


,age,sex,bmi,children,smoker,region,charges
0,19,0.0,27.900,0,1.0,3.0,16884.92400
1,18,1.0,33.770,1,0.0,2.0,1725.55230
2,28,1.0,33.000,3,0.0,2.0,4449.46200
3,33,1.0,22.705,0,0.0,1.0,21984.47061
4,32,1.0,28.880,0,0.0,1.0,3866.85520
5,31,0.0,25.740,0,0.0,2.0,3756.62160
6,46,0.0,33.440,1,0.0,2.0,8240.58960
7,37,0.0,27.740,3,0.0,1.0,7281.50560
8,37,1.0,29.830,2,0.0,0.0,6406.41070
9,60,0.0,25.840,0,0.0,1.0,28923.13692


### Normalisation des variables numériques
Normaliser les variables numériques pour les mettre à l'échelle en utilisant `StandardScaler` ou `MinMaxScaler`

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

numerical_column = ['age','bmi','children']
# scaled_features = scaler.fit_transform(df[numerical_column])
#
# scaled_df = pd.DataFrame(scaled_features, columns=numerical_column)

df[numerical_column] = scaler.fit_transform(df[numerical_column])
print(df.head(10))


        age  sex       bmi  children  smoker  region      charges
0 -1.438764  0.0 -0.453320 -0.908614     1.0     3.0  16884.92400
1 -1.509965  1.0  0.509621 -0.078767     0.0     2.0   1725.55230
2 -0.797954  1.0  0.383307  1.580926     0.0     2.0   4449.46200
3 -0.441948  1.0 -1.305531 -0.908614     0.0     1.0  21984.47061
4 -0.513149  1.0 -0.292556 -0.908614     0.0     1.0   3866.85520
5 -0.584350  0.0 -0.807656 -0.908614     0.0     2.0   3756.62160
6  0.483668  0.0  0.455486 -0.078767     0.0     2.0   8240.58960
7 -0.157143  0.0 -0.479567  1.580926     0.0     1.0   7281.50560
8 -0.157143  1.0 -0.136714  0.751079     0.0     0.0   6406.41070
9  1.480485  0.0 -0.791252 -0.908614     0.0     1.0  28923.13692


### Construction du dataset final
Assembler les variables encodées et normalisées pour créer la matrice des caractéristiques `X` et la variable cible `y`.

In [ ]:
X = df.drop('charges', axis=1)
y = df['charges']
print(X.head())
print(y.head())

        age  sex       bmi  children  smoker  region
0 -1.438764  0.0 -0.453320 -0.908614     1.0     3.0
1 -1.509965  1.0  0.509621 -0.078767     0.0     2.0
2 -0.797954  1.0  0.383307  1.580926     0.0     2.0
3 -0.441948  1.0 -1.305531 -0.908614     0.0     1.0
4 -0.513149  1.0 -0.292556 -0.908614     0.0     1.0
0    16884.92400
1     1725.55230
2     4449.46200
3    21984.47061
4     3866.85520
Name: charges, dtype: float64


# 5. Division en ensembles d'entraînement et de test
Séparer les données en un ensemble d'entraînement (80%) et un ensemble de test (20%).

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

# 6. Implémentation d'un modèle de régression linéaire
Créer et entraîner un modèle de régression linéaire.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression

In [ ]:
clf = LinearRegression()
clf.fit(X_train, y_train)


y_pred_test = clf.predict(X_test)

for i in range(len(y_pred_test)):
    print(str(list(y_pred_test)[i]) + ' vs ' + str(list(y_test)[i]))


ValueError: Unknown label type: continuous. Maybe you are trying to fit a classifier, which expects discrete classes on a regression target with continuous values.

# 7. Prédictions et évaluation du modèle linéaire
Évaluer les performances du modèle avec l'erreur quadratique moyenne (MSE) et le coefficient de détermination (R²) sur le dataset d'entrainement et de test. Que pensez-vous de la qualité de ce modèle?

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

y_pred_train = clf.predict(X_train)
y_pred_test = clf.predict(X_test)

print(f"Train MSE = {mean_squared_error(y_train, y_pred_train)}")
print(f"Test MSE = {mean_squared_error(y_test, y_pred_test)}")



Train MSE = 34235485.624779925
Test MSE = 43518206.188537404
Train R² Score = 0.7574927230168875
Test R² Score = 0.7312777518855209


# 8. Implémentation d'un modèle KNN
Créer, entraîner et évaluer un modèle KNN avec `k=5`.

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
knc = KNeighborsRegressor(5)
knc.fit(X_train, y_train)
y_pred_train = knc.predict(X_train)
y_pred_test = knc.predict(X_test)
print(f"Train MSE = {mean_squared_error(y_train, y_pred_train)}")
print(f"Test MSE = {mean_squared_error(y_test, y_pred_test)}")
print(f"Train R² Score = {r2_score(y_train, y_pred_train)}")
print(f"Test R² Score = {r2_score(y_test, y_pred_test)}")

Train MSE = 29498468.082138244
Test MSE = 42818670.30661694
Train R² Score = 0.7980351207931636
Test R² Score = 0.7103611897819913


# 9. Étude de l'impact du nombre de voisins
Tester l'effet du nombre de voisins sur la performance du modèle KNN.
Vous devez produire un graphique qui affiche la MSE en ordonnées en fonction du nombre voisin (en abscisse). Les valeurs en entrainement et en test doivent être affichées

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
knc = KNeighborsRegressor(3)
knc.fit(X_train, y_train)
y_pred_train = knc.predict(X_train)
y_pred_test = knc.predict(X_test)
print(f"Train MSE = {mean_squared_error(y_train, y_pred_train)}")
print(f"Test MSE = {mean_squared_error(y_test, y_pred_test)}")
print(f"Train R² Score = {r2_score(y_train, y_pred_train)}")
print(f"Test R² Score = {r2_score(y_test, y_pred_test)}")


Train MSE = 21504359.09939896
Test MSE = 40337845.14895731
Train R² Score = 0.8527677682841988
Test R² Score = 0.7271422631286897


In [ ]:
from sklearn.neighbors import KNeighborsRegressor
knc = KNeighborsRegressor(7)
knc.fit(X_train, y_train)
y_pred_train = knc.predict(X_train)
y_pred_test = knc.predict(X_test)
print(f"Train MSE = {mean_squared_error(y_train, y_pred_train)}")
print(f"Test MSE = {mean_squared_error(y_test, y_pred_test)}")
print(f"Train R² Score = {r2_score(y_train, y_pred_train)}")
print(f"Test R² Score = {r2_score(y_test, y_pred_test)}")

Train MSE = 36145406.75628496
Test MSE = 47336770.93821693
Train R² Score = 0.7525260400272996
Test R² Score = 0.6797993511725464


# 10. Analyse des résultats
- Quelle est l'importance des différentes variables dans la prédiction ?
- Quel est le meilleur choix pour le paramètre `k` ?

# 11. Pour les plus rapides ou à faire à la maison
Améliorer l'analyse de paramètres précédentes pour répeter plusieurs fois les entraintements sur des splits train/test différents. Afficher sur le plot la moyenne et les écart-types obtenus sur 20 splits différents.

# 12. Autres modèles
Tester d'autres modèles de regression linéaire "simples" tels que des modèles polynomiaux, Lasso, ElasticNet ou Ridge